In [16]:
import Tools
import pandas as pd
import index_generate
from PIL import Image
import os

#去除因FutureWarning 而無法顯示dataframe的bug，要執行兩次CELL ，我不知道怎麼解決...
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5)) #returns False, without Warning

class buyer():
    state = 0
    fund = 2000000
    stock = 0
    total = fund + stock
    stockcode = '0'

    indexflag=0
    df_raw=pd.DataFrame()
    df_index=pd.DataFrame()
    min_maxdf=pd.DataFrame()

    df_index_V2=pd.DataFrame()
    min_maxdf_V2=pd.DataFrame()



    List_ans=np.array(0)
    List_ans_V3=np.array(0)
    List_price=np.array(0)


    def __init__(self,stockcode):
        self.stockcode=stockcode
        self.df_raw,self.indexflag =Tools.checkCodeInDir(self.stockcode) #檢查是否在檔案中，並取得rawdata

        if(self.indexflag==0):
            self.df_index=index_generate.stock_index_generator(self.df_raw,self.stockcode)
        else:
            PATH_TO_STOCKDATA_INDEX = './stock_data_index/'
            index_target = stockcode+'_index.json'
            self.df_index = pd.read_json(PATH_TO_STOCKDATA_INDEX+index_target)

        self.min_maxdf=index_generate.Normalize_pd(self.df_index)# 計算Normalize

        self.df_index_V2=index_generate.stock_index_generator_V2(self.df_index,self.stockcode)
        self.min_maxdf_V2=index_generate.Normalize_pd_V2(self.df_index_V2)# 計算Normalize

        self.List_ans, self.List_price,self.List_ans_V3=Tools.get_list_ans(self.stockcode)#計算label答案
    
    def getImg(self,index): #取得index 當天加上前18筆 共19筆資料的圖片
        v=self.min_maxdf.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=19
        img = Image.fromarray(v[index-traceback_Date:index,:],'F')
        img = img.convert('RGB')
        return img

    def saveImg(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=19
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output\\'+self.stockcode+'\\'+str(self.List_ans[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output/'+self.stockcode+'/'+str(self.List_ans[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
        
    def getImg_V2(self,index): #取得index 當天加上前18筆 共19筆資料的圖片
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=11
        img = Image.fromarray(v[index-traceback_Date:index,:],'F')
        img = img.convert('RGB')
        return img

    def saveImg_V2(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=11
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output_V2\\'+self.stockcode+'\\'+str(self.List_ans[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output_V2/'+self.stockcode+'/'+str(self.List_ans[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
    def getImg_V3(self,index): #取得index 當天加上前18筆 共19筆資料的圖片
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=3
        img = Image.fromarray(v[index-traceback_Date:index,:],'F')
        img = img.convert('RGB')
        return img

    def saveImg_V3(self,index):#取得index 當天加上前18筆 共19筆資料的圖片，並自動存檔
        v=self.min_maxdf_V2.to_numpy()
        v=v.astype(np.float32)
        traceback_Date=3
        img=Image.fromarray(v[index-traceback_Date:index,:],'F')
        date=str(self.df_raw['日期'][index]).replace("/","")
                
        DIRPATH=os.path.abspath('.')+'\image_output_V3\\'+self.stockcode+'\\'+str(self.List_ans_V3[index])
        try:
            os.makedirs(DIRPATH)
        except OSError:
            pass

        pwd=os.path.abspath('.').replace("\\","/")
        pwd=pwd.replace("c:/","C:/")
        PATH=pwd+'/image_output_V3/'+self.stockcode+'/'+str(self.List_ans_V3[index])+'/'
        PATH=PATH.replace(" ","")
        print(PATH)

        img = img.convert('RGB')
        img.save(PATH+date+'.jpg',"JPEG")
        return img
    
    
    def get_ans_price(self):
        return self.List_ans, self.List_price

    def buystock(self,price):
        self.stock=(self.fund/1.003)/price
        self.fund=0
        self.total=self.fund+(self.stock*price)
        self.state=1
        print("buy")
    def selstock(self,price):   
        self.fund=self.stock*price*0.997
        self.stock=0
        self.total=self.fund+(self.stock*price)
        self.state=0
        print("sell")
    def holstock(self,price):
        self.total=self.fund+(self.stock*price)
        print("hold")
    def make_choice(self,inp,price):
        if (self.state==0 and inp==1):
            self.buystock(price)
        elif(self.state==1 and inp==-1):
            self.selstock(price)
        else :
            self.holstock(price)

False


In [5]:
Tom=buyer('0050')
for i in range(Tom.List_ans.size-1):
    Tom.make_choice(Tom.List_ans_V3[i],float(Tom.List_price[i]))
print(Tom.total)

found data [0050.json]at : ./stock_data
found data [0050_index.json]at : ./stock_data_index


D:\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
buy
sell
hold
hold
buy
hold
hold
sell
hold
hold
hold
buy
hold
hold
sell
hold
hold
buy
hold
hold
hold
sell
hold
buy
hold
hold
hold
sell
buy
hold
sell
buy
hold
sell
hold
buy
hold
sell
buy
hold
sell
hold
hold
hold
hold
buy
sell
hold
buy
hold
sell
hold
buy
sell
hold
buy
sell
hold
hold
buy
sell
buy
hold
hold
hold
sell
hold
buy
sell
hold
hold
hold
buy
hold
hold
hold
hold
sell
buy
sell
hold
buy
sell
buy
sell
hold
hold
buy
hold
hold
sell
buy
hold
sell
hold
buy
hold
sell
hold
buy
sell
buy
hold
hold
sell
buy
hold
sell
buy
hold
sell
hold
buy
hold
sell
hold
hold
buy
hold
sell
hold
buy
hold
hold
sell
hold
hold
buy
hold
sell
buy
hold
hold
hold
sell
hold
buy
hold
hold
hold
hold
sell
buy
sell
buy
sell
hold
buy
sell
buy
sell
buy
sell
hold
buy
hold
sell
hold
hold
buy
hold
sell
hold
hold
buy
hold
hold
sell
buy
sell
buy
sell
buy
sell
hold
buy
hold
sell
buy
sell
hold
hold
hold
buy
sell
buy
hold
hold
sell
hold
buy

In [ ]:
# test input get img
# 毛哥要用的時候
# cnn_ans(img)換成你寫好的預測結果

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
import torchvision.models as models

CUDA_DEVICES=0
PATH_TO_WEIGHTS = './Model_all_best.pth'

def cnn_ans(img):
    
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    model=models.resnet101()
    model = torch.load(PATH_TO_WEIGHTS)
    model = model.cuda(CUDA_DEVICES)
    model.eval()
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():

        # data_loader的格式 (batch_size, channels, height, width)
        # 因為這裡沒用 data_loader() 當媒介餵資料，所以缺少batch維度，要用unsqueeze在最前面增加一個維度。
        
        inputs = data_transform(img).unsqueeze(0)
        inputs = Variable(inputs.cuda())  # CUDA_DEVICES))

        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        tmp = outputs.data.cpu().numpy()
        _, predictions = torch.max(outputs.data, 1)
        
        change_ans = [-1,0,1]
        
        predictions = change_ans[predictions.int()]
               
        print("Predictions: ", predictions)
    
    
    return predictions


John=buyer('0056')
for i in range(19,John.List_ans.size-1):
    img=John.getImg(i)
    ans=cnn_ans(img)
    John.make_choice(ans,float(John.List_price[i]))
print(John.total)


found data [0056.json]at : ./stock_data
found data [0056_index.json]at : ./stock_data_index
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  1
buy
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  1
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predictions:  0
hold
Predic

In [2]:
# test input savelabel

Bob=buyer('0053')
Bob.min_maxdf

for i in range(19,Bob.List_ans.size-1):
    img=Bob.saveImg(i)
    Bob.make_choice(Bob.List_ans[i],float(Bob.List_price[i]))



ing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output/0053/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output/0053/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output/0053/0/
hold
C:/Users/zing9/Graphics-in-Financ

In [17]:
Wei=buyer('0050')
for i in range(60,Wei.List_ans.size-1):
    img=Wei.saveImg_V3(i)
    Wei.make_choice(Wei.List_ans_V3[i],float(Wei.List_price[i]))

found data [0050.json]at : ./stock_data
found data [0050_index.json]at : ./stock_data_index
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graph

sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output

C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3

C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/005

C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0

C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/005

sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_outpu

C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/005

C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/005

hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
sell
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/-1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
buy
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_output_V3/0050/1/
hold
C:/Users/user/Graphics-in-Finance/image_outpu

In [8]:
Bob=buyer('0054')

for i in range(19,Bob.List_ans.size-1):
    img=Bob.saveImg_V2(i)
    Bob.make_choice(Bob.List_ans[i],float(Bob.List_price[i]))


image_output_V2/0054/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/1/
buy
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/-1/
sell
C:/Users/zing9/Graphics-in-Finance/image_output_V2/0054/0/
hold
C:/Users/zi